In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa


In [2]:
TRAINING_DIR = 'F:/Xray_Gaus/train'
TEST_DIR='F:/Xray_Gaus/test'
VAL_DIR='F:/Xray_Gaus/val'


In [3]:
num_classes = 2
IMG_SIZE = 224
BATCH_SIZE = 32
input_shape = (224,224, 3)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 10

In [4]:
from keras_preprocessing.image import ImageDataGenerator

# Training IDG
train_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
train_gen = train_idg.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_175','COVID-19'],
    subset='training'
) 

Found 350 images belonging to 2 classes.


In [5]:
val_idg = ImageDataGenerator(
    rescale = 1./255, 
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.1, 
    zoom_range=0.1,
    horizontal_flip=True)

# Training Gen
val_gen = val_idg.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='binary',
    classes=['normal_20','COVID-19'],
    subset='training'
)

Found 40 images belonging to 2 classes.


In [6]:
test_idg = ImageDataGenerator(rescale=1./255)

# Test Gen
test_gen = test_idg.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        class_mode='binary',
    classes=['normal_20','COVID-19'],
        #subset='validation'
)

Found 40 images belonging to 2 classes.


In [7]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

from tensorflow.keras.optimizers import Adam

base_model = keras.applications.VGG19(
    weights = 'imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

x = base_model.output
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

# se acopla el modelo
model = keras.Model(base_model.input, outputs)

# congelar capas
for layer in base_model.layers:
    layer.trainable = False

# compilar el modelo.
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')]
        )

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [15]:
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = val_gen.n // val_gen.batch_size

history=model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = 2)

Epoch 1/2
10/10 [==============================] - 107s 11s/step - loss: 0.3724 - accuracy: 0.9182 - precision: 0.9125 - recall: 0.9241 - val_loss: 5.0164 - val_accuracy: 0.6875 - val_precision: 1.0000 - val_recall: 0.4118
Epoch 2/2
10/10 [==============================] - 101s 10s/step - loss: 0.3306 - accuracy: 0.9151 - precision: 0.9177 - recall: 0.9119 - val_loss: 2.7006 - val_accuracy: 0.8125 - val_precision: 1.0000 - val_recall: 0.6000


In [22]:
model.save('F:/X_ray_Models/vgg19_xray_model_2')

history.history

INFO:tensorflow:Assets written to: F:/X_ray_Models/vgg19_xray_model_2\assets


{'loss': [0.3723961114883423, 0.33056411147117615],
 'accuracy': [0.9182389974594116, 0.9150943160057068],
 'precision': [0.9125000238418579, 0.9177215099334717],
 'recall': [0.9240506291389465, 0.9119496941566467],
 'val_loss': [5.0164055824279785, 2.700631618499756],
 'val_accuracy': [0.6875, 0.8125],
 'val_precision': [1.0, 1.0],
 'val_recall': [0.4117647111415863, 0.6000000238418579]}

In [23]:
results = model.evaluate(test_gen, batch_size=10)

2/2 [==============================] - 9s 2s/step - loss: 1.3595 - accuracy: 0.8250 - precision: 1.0000 - recall: 0.6500


In [24]:
y_pred = model.predict(test_gen, verbose=1)

#y_pred = np.argmax(y_pred, axis = 1)


for i in range(len(y_pred)):
    if y_pred[i]>=0.5:
        y_pred[i]=1
    else:
        y_pred[i]=0

2/2 [==============================] - 9s 2s/step


In [25]:
y_true=test_gen.labels

In [26]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.44      0.60      0.51        20
           1       0.38      0.25      0.30        20

    accuracy                           0.42        40
   macro avg       0.41      0.42      0.41        40
weighted avg       0.41      0.42      0.41        40



In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

array([[12,  8],
       [15,  5]], dtype=int64)